In [1]:
reset -fs

In [5]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [6]:
RANDOM_STATE = 28

In [7]:
# Comments File
comments = '/Users/jacquessham/Documents/MSDS/MSAN621/FinalProject/Dataset/combined2.csv'
# Read in File
comm = pd.read_csv(comments)
filt = (comm.sectionName != "Unknown") & (comm.commentType == "comment")
unknown = (comm.sectionName != "Unknown") & (comm.commentType == "comment")
data = comm.commentBody[filt]
sections = comm.sectionName[filt]
data_unknown = comm.commentBody[unknown]
sections_unknown = comm.sectionName[unknown]
train_data, test_data, train_target, test_target = train_test_split(data, sections, random_state=RANDOM_STATE)

In [8]:
from sklearn.ensemble import AdaBoostClassifier

vectorizer = CountVectorizer(decode_error='ignore',
                             stop_words='english')
booster_NB_model = Pipeline([('vec', vectorizer),
                    ('boo', AdaBoostClassifier(MultinomialNB(alpha=0.30000000000000004)))])

In [9]:
booster_NB_model.fit(train_data, train_target)

Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='word', binary=False, decode_error='ignore',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        s...lass_prior=None, fit_prior=True),
          learning_rate=1.0, n_estimators=50, random_state=None))])

In [10]:
booster_NB_model.score(test_data, test_target)

0.5548219626431917

Boosting with Naive Bayes gave me 0.5548 accuracy

In [11]:
from sklearn.svm import LinearSVC

booster_SVM_model = Pipeline([('vec', vectorizer),
                              ('svm', LinearSVC(random_state=RANDOM_STATE, loss='hinge'))])
booster_SVM_model.fit(train_data, train_target)

/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='word', binary=False, decode_error='ignore',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        s...nge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=28, tol=0.0001, verbose=0))])

In [12]:
booster_SVM_model.score(test_data, test_target)

0.7271880819366853

SVM with boosting gave me 0.7272 accuracy

In [13]:
from sklearn.ensemble import GradientBoostingClassifier

vectorizer = CountVectorizer(decode_error='ignore',
                             stop_words='english')
gbooster_NB_model = Pipeline([('vec', vectorizer),
                    ('gbo', GradientBoostingClassifier())])
gbooster_NB_model.fit(train_data, train_target)
gbooster_NB_model.score(test_data, test_target)

0.6995937023869985

It gave me 0.6996 accuracy, it did not give bad accuracy as worst as Naive Bayes boosting

Here to try Perceptron <= But don't know what is it

In [14]:
from sklearn.linear_model import Perceptron

vectorizer = CountVectorizer(decode_error='ignore',
                             stop_words='english')
pet_model = Pipeline([('vec', vectorizer),
                    ('pet', Perceptron())])
pet_model.fit(train_data, train_target)
pet_model.score(test_data, test_target)

/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.6662152248744427

It gave me 0.6662
Didn't work too well
Try Perceptron with booster

In [15]:
from sklearn.ensemble import AdaBoostClassifier

booster_pet_model = Pipeline([('vec', vectorizer),
                    ('bpt', AdaBoostClassifier(Perceptron(), algorithm='SAMME'))])
booster_pet_model.fit(train_data, train_target)
booster_pet_model.score(test_data, test_target)

0.6270526493990182

It gave me 0.6229

In [16]:
from sklearn.linear_model import SGDClassifier

vectorizer = CountVectorizer(decode_error='ignore',
                             stop_words='english')
sgd_model = Pipeline([('vec', vectorizer),
                    ('sgd', SGDClassifier())])
sgd_model.fit(train_data, train_target)
sgd_model.score(test_data, test_target)

/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.7373173071497093

It gave me 0.7376
This code ran for more than 30 mins
SVG has slightly higher score than NB inintally, it is worth to grid search on this.

In [18]:
from sklearn.model_selection import GridSearchCV

grid_params = dict(vec__ngram_range=[(1,1), (1,2), (1,3)],
                   sgd__loss=['hinge', 'log', 'modified_huber', 'squared_hinge'])
gs = GridSearchCV(estimator=sgd_model,
                 param_grid=grid_params,
                 scoring='accuracy',
                 cv=5)
gs.fit(train_data, train_target)
gs.best_params_

/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/model_selection/_split.py:626: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


{'sgd__loss': 'hinge', 'vec__ngram_range': (1, 2)}

In [19]:
gs.score(test_data, test_target)

0.744399300265222

It gives a good accuracy but still worse than Naive Bayes